In [ ]:
!pip install torch==1.7.1
!pip install transformers==4.11.3
!pip install huggingface-hub==0.0.19
!pip install datasets==1.5.0

In [1]:
import json
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from tqdm import trange

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    BertModel, RobertaModel,
    BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments,
)
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from typing import List
from torch.utils.data import Sampler

In [2]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_seed(42) # magic number :)

In [3]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.7.1].
device:[cuda:0].


## Training

In [4]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
        classifier_dropout=(
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = torch.nn.Dropout(classifier_dropout)
        self.linear = torch.nn.Linear(config.hidden_size, 1)
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        output = self.linear(pooled_output)
        return output

In [5]:
dataset = load_from_disk('/opt/ml/data/train_dataset')
train_dataset = dataset['train']

In [6]:
class CustomSampler(Sampler) :
    def __init__(self, data_source, batch_size) :
        self.data_source = data_source
        self.batch_size = batch_size

    def __iter__(self) :
        n = len(self.data_source)
        index_list = []
        while True :
            out = True
            for i in range(self.batch_size) :
                tmp_data = random.randint(0, n-1)
                index_list.append(tmp_data)
            for f, s in zip(index_list, index_list[1:]) :
                if abs(s-f) <= 2 :
                    out = False
            if out == True :
                break

        while True : # 추가 삽입
            tmp_data = random.randint(0, n-1)
            if (tmp_data not in index_list) and \
                (abs(tmp_data-index_list[-i]) > 2 for i in range(1,self.batch_size+1)) \
            : 
                index_list.append(tmp_data)
            if len(index_list) == n :
                break
        return iter(index_list)

    def __len__(self) :
        return len(self.data_source)

In [7]:
# Anwer
class DenseRetrieval:
    def __init__(self,
        args,
        dataset,
        tokenizer,
        cross_encoder,
        sampler
    ):
        """
        학습과 추론에 사용될 여러 셋업을 마쳐봅시다.
        """

        self.args = args
        self.dataset = dataset

        self.tokenizer = tokenizer
        self.cross_encoder = cross_encoder
        self.sampler = sampler

    def train(self, args=None, tokenizer = None):
        if args is None:
            args = self.args
        if tokenizer is None :
            tokenizer = self.tokenizer
        
        tokenized_examples = tokenizer(
            self.dataset['question'],
            self.dataset['context'],
            truncation="only_second",
            max_length=512,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            # return_token_type_ids=False,  # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
            padding="max_length",
            return_tensors='pt'
        )

        train_dataset = TensorDataset(
            tokenized_examples['input_ids'],
            tokenized_examples['attention_mask'],
            tokenized_examples['token_type_ids']
        )

        sampler = self.sampler(train_dataset, args.per_device_train_batch_size)
        train_dataloader = DataLoader(train_dataset,
                                      batch_size=args.per_device_train_batch_size,
                                      sampler = sampler,
                                      drop_last = True)
                                      
        no_decay = ["bias" ,"LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {"params": [p for n, p in self.cross_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.cross_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
        ]
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            # eps=args.adam_epsilon
        )

        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)
        
        self.cross_encoder.zero_grad()
        
        train_iterator = trange(int(args.num_train_epochs), desc="Epoch")
        self.cross_encoder.train()
        for epoch, _ in enumerate(train_iterator) :
            epoch_iterator = tqdm(train_dataloader, desc = 'Iteration')
            losses = 0
            for step, batch in enumerate(epoch_iterator) :
                # if torch.cuda.is_available() :
                #     batch = tuple(t.cuda() for t in batch)
                
                cross_inputs = {
                    'input_ids': batch[0],
                    'attention_mask' : batch[1],
                    'token_type_ids' : batch[2]
                }
                for k in cross_inputs.keys() :
                    cross_inputs[k] = cross_inputs[k].tolist()

                new_input_ids = []
                new_attention_mask = []
                new_token_type_ids = []
                for i in range(len(cross_inputs['input_ids'])) :
                    sep_index = cross_inputs['input_ids'][i].index(3) # [SEP] token의 index

                    for j in range(len(cross_inputs['input_ids'])) :
                        query_id = cross_inputs['input_ids'][i][:sep_index]
                        query_att = cross_inputs['attention_mask'][i][:sep_index]
                        query_tok = cross_inputs['token_type_ids'][i][:sep_index]
        
                        context_id = cross_inputs['input_ids'][j][sep_index:]
                        context_att = cross_inputs['attention_mask'][j][sep_index:]
                        context_tok = cross_inputs['token_type_ids'][j][sep_index:]
                        query_id.extend(context_id)
                        query_att.extend(context_att)
                        query_tok.extend(context_tok)
                        new_input_ids.append(query_id)
                        new_attention_mask.append(query_att)
                        new_token_type_ids.append(query_tok)

                change_cross_inputs = {
                    'input_ids' : torch.tensor(new_input_ids).to('cuda'),
                    'attention_mask' : torch.tensor(new_attention_mask).to('cuda'),
                    'token_type_ids' : torch.tensor(new_token_type_ids).to('cuda')
                }

                cross_output = self.cross_encoder(**change_cross_inputs)
                cross_output = cross_output.view(-1, args.per_device_train_batch_size)
                targets = torch.arange(0, args.per_device_train_batch_size).long()
                                
                if torch.cuda.is_available():
                    targets = targets.to('cuda')

                score = F.log_softmax(cross_output, dim = 1)
                loss = F.nll_loss(score, targets)
                
                losses += loss.item()
                if step % 100 == 0 :
                    print(f'{epoch}epoch loss: {losses/(step+1)}') # Accumulation할 경우 주석처리
                
                self.cross_encoder.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()
        
        return self.cross_encoder

In [8]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    num_train_epochs=5,
    weight_decay=0.01
)
model_checkpoint = "klue/bert-base"

# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
cross_encoder = BertEncoder.from_pretrained(model_checkpoint).to('cuda')

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertEncoder were not initialized from the model checkpoint at klue/bert-base and are newly initialized: 

In [9]:
# Retriever는 아래와 같이 사용할 수 있도록 코드를 짜봅시다.
retriever = DenseRetrieval(
    args=args,
    dataset=train_dataset,
    tokenizer=tokenizer,
    cross_encoder=cross_encoder,
    sampler = CustomSampler
)
c_encoder = retriever.train()

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

0epoch loss: 1.1035704612731934
0epoch loss: 0.18624336721541562
0epoch loss: 0.10813956529675839
0epoch loss: 0.0786655272499881
0epoch loss: 0.061980607657823386
0epoch loss: 0.05149939898963824
0epoch loss: 0.04353808492237893
0epoch loss: 0.03798551275547434
0epoch loss: 0.03382088217534439
0epoch loss: 0.030704726455816195
0epoch loss: 0.028170246483781525
0epoch loss: 0.02641521077222254
0epoch loss: 0.024651965653075203
0epoch loss: 0.022974016423186878
0epoch loss: 0.021735249884547246


Epoch:  20%|██        | 1/5 [12:34<50:19, 754.85s/it]

1epoch loss: 5.0275000830879435e-05
1epoch loss: 0.005915985919669441
1epoch loss: 0.003613268038649193
1epoch loss: 0.0026920617997302694
1epoch loss: 0.002090122697800983
1epoch loss: 0.0025501002893540503
1epoch loss: 0.002366891297763122
1epoch loss: 0.0021212439260073494
1epoch loss: 0.00194129908855773
1epoch loss: 0.0017827807080122368
1epoch loss: 0.0016300902424283103
1epoch loss: 0.0015002492954128773
1epoch loss: 0.0016055718124293166
1epoch loss: 0.001540191611271861
1epoch loss: 0.001608899648548943


Epoch:  40%|████      | 2/5 [25:10<37:45, 755.00s/it]

...

Epoch:  60%|██████    | 3/5 [37:45<25:09, 754.94s/it]

3epoch loss: 0.0004370314709376544
3epoch loss: 0.0002799427811939072
3epoch loss: 0.002573312194297035
3epoch loss: 0.0025495006939104995
3epoch loss: 0.0020428936650939574
3epoch loss: 0.0018965851498055082
3epoch loss: 0.0027036759014070944
3epoch loss: 0.0023838151372386524
3epoch loss: 0.0025146828743072887
3epoch loss: 0.0023959424109831692
3epoch loss: 0.002415488747367142
3epoch loss: 0.0022140688547954398
3epoch loss: 0.0025566949302962133
3epoch loss: 0.002432697453730541
3epoch loss: 0.0022708003295246887


Epoch:  80%|████████  | 4/5 [50:20<12:34, 754.99s/it]

4epoch loss: 2.2649737729807384e-06
4epoch loss: 0.0004296058863674531
4epoch loss: 0.0018704596437794578
4epoch loss: 0.0013476845011312214
4epoch loss: 0.001074609675461623
4epoch loss: 0.0008875762978842666
4epoch loss: 0.0007734266130895952
4epoch loss: 0.0006721240544348424
4epoch loss: 0.0006399781945218026
4epoch loss: 0.0005796041422958585
4epoch loss: 0.0005411629326906543
4epoch loss: 0.000566605232685894
4epoch loss: 0.000902799399744505
4epoch loss: 0.0009050748027463063
4epoch loss: 0.0008532462466982899


Epoch: 100%|██████████| 5/5 [1:02:54<00:00, 754.99s/it]

In [13]:
torch.save(c_encoder, '/opt/ml/custom/c_encoder_e5.pt')

## 실험

In [17]:
valid_corpus = list(set([example['context'] for example in dataset['validation']]))[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(query)
print(ground_truth)

동맹항에서의 교역에서 세금을 내지 않으려면 지참해야 하는 것은 무엇인가?
:\n; 교역\n:* 첫 번째 시리즈처럼, 어느 도시에서 사들인 상품을 다른 도시로 수송해, 그 도시에서 다른 상품을 구매한 다음, 처음 상품을 샀던 도시로 가서 상품을 매각하면서 수익을 올려 가는 것이 초반의 기본 방식이다. 면세증을 가지고 있으면 해당 국가의 동맹항에서 교역할 때는 세금이 붙지 않는다. 주인공이 회계 또는 교섭 능력이 있거나 회계 능력을 가지고 있는 항해사를 경리주임에 임명하면 상품 구입 시 가격흥정이 가능하다.\n:\n; 조선\n:* 파손된 배 수리, 중고선 매매, 선수상이나 대포 설치, 함선 건조를 할 수 있다. 투자를 반복해서 상업가치나 공업가치가 최대치가 되면 진귀한 선수상(천사·여신)이나 대포(카로네이드포) 구입이 가능하며, 일부 항구의 조선소에서는 특수한 배(바그, 프리게이트, 철갑선, 쉽)의 건조도 가능하다.\n:\n; 투자\n:* 항구에 있는 교역소나 조선소에 투자를 하면, 다음달에 그 도시의 상업가치나 공업가치가 올라 교역소에 새로운 상품이 등장하거나 조선소에서 대형함선을 건조할 수 있다. 또한, 자국에 대한 지지율이 올라, 지지율이75%이상이 되면 그 항구는 자국의 동맹항이 된다. 단, 각국의 수도(리스본, 세빌리아, 런던, 암스테르담, 제노바, 이스탄불)에는 투자할 수 없다. 대항해시대에는 전 세계에 총 100개의 항구가 존재하는데, 여기서 각국의 수도를 제외하고 나면 자국의 동맹항으로 할 수 있는 최대 항구수는 95개 (나머지 항구 94개 + 자국의 수도) 가 된다.\n:\n;해전\n:* 해적이나 다른 함대를 습격할 수 있다. 기본은 헥스(HEX)방식의 전략 시뮬레이션이지만, 기함끼리 인접하면 제독의 일기토로 승부를 붙이는 것이 가능하다.(자함대 갑판에 편성된 선원수가 타함대 갑판선원수보다 많이 부족한 경우에는 불가). 승리하면 적함대의 함선, 적하, 보물을 빼앗을 수 있다. 해적 명성이 일정치 이상이며, 모험이나 교역 명성보다 높은 경우, 자국의

In [26]:
with torch.no_grad() :
    c_encoder.eval()
    
    score_list = []
    for i in range(len(valid_corpus)) :
        passage = valid_corpus[i]
        tokenized_examples = tokenizer(
            query,
            passage,
            truncation="only_second",
            max_length=512,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            #return_token_type_ids=False,  # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
            padding="max_length",
            return_tensors='pt'
        )

        score = 0
        for i in range(len(tokenized_examples['input_ids'])) :
            c_input = {
                'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
                'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
                'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')
            }
            tmp_score = c_encoder(**c_input).to('cpu')
            score += tmp_score
        score = score / len(tokenized_examples['input_ids'])
        score_list.append(score)
    sort_result = torch.sort(torch.tensor(score_list), descending=True)

    scores, index_list = sort_result[0], sort_result[1]

<ipython-input-26-411eda001f16>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
<ipython-input-26-411eda001f16>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
<ipython-input-26-411eda001f16>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')


In [29]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, scores[i]))
  print(valid_corpus[index_list[i]])

[Search query]
 동맹항에서의 교역에서 세금을 내지 않으려면 지참해야 하는 것은 무엇인가? 

[Ground truth passage]
:\n; 교역\n:* 첫 번째 시리즈처럼, 어느 도시에서 사들인 상품을 다른 도시로 수송해, 그 도시에서 다른 상품을 구매한 다음, 처음 상품을 샀던 도시로 가서 상품을 매각하면서 수익을 올려 가는 것이 초반의 기본 방식이다. 면세증을 가지고 있으면 해당 국가의 동맹항에서 교역할 때는 세금이 붙지 않는다. 주인공이 회계 또는 교섭 능력이 있거나 회계 능력을 가지고 있는 항해사를 경리주임에 임명하면 상품 구입 시 가격흥정이 가능하다.\n:\n; 조선\n:* 파손된 배 수리, 중고선 매매, 선수상이나 대포 설치, 함선 건조를 할 수 있다. 투자를 반복해서 상업가치나 공업가치가 최대치가 되면 진귀한 선수상(천사·여신)이나 대포(카로네이드포) 구입이 가능하며, 일부 항구의 조선소에서는 특수한 배(바그, 프리게이트, 철갑선, 쉽)의 건조도 가능하다.\n:\n; 투자\n:* 항구에 있는 교역소나 조선소에 투자를 하면, 다음달에 그 도시의 상업가치나 공업가치가 올라 교역소에 새로운 상품이 등장하거나 조선소에서 대형함선을 건조할 수 있다. 또한, 자국에 대한 지지율이 올라, 지지율이75%이상이 되면 그 항구는 자국의 동맹항이 된다. 단, 각국의 수도(리스본, 세빌리아, 런던, 암스테르담, 제노바, 이스탄불)에는 투자할 수 없다. 대항해시대에는 전 세계에 총 100개의 항구가 존재하는데, 여기서 각국의 수도를 제외하고 나면 자국의 동맹항으로 할 수 있는 최대 항구수는 95개 (나머지 항구 94개 + 자국의 수도) 가 된다.\n:\n;해전\n:* 해적이나 다른 함대를 습격할 수 있다. 기본은 헥스(HEX)방식의 전략 시뮬레이션이지만, 기함끼리 인접하면 제독의 일기토로 승부를 붙이는 것이 가능하다.(자함대 갑판에 편성된 선원수가 타함대 갑판선원수보다 많이 부족한 경우에는 불가). 승리하면 적함대의 함선, 적하, 보물을 빼앗을 수 있다.

In [28]:
index_list

tensor([10,  0,  4,  1,  5,  9,  6,  3,  7,  2,  8])

In [27]:
tokenized_examples['input_ids'][0].unsqueeze(dim=0).shape

torch.Size([1, 512])

In [ ]:
top_k_index_list = []
for i in range(len(index_list)) :
    temp = index_list[i][:k]
    top_k_index_list.appedn(temp)

## 실제

In [30]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로

In [32]:
question_data = dataset['validation']['question']
with torch.no_grad() :
    c_encoder.eval()

    result_scores = []
    result_indices = []
    for i in tqdm(range(len(question_data))) :
        question = question_data[i]

        question_score = []
        for i in tqdm(range(len(corpus))) :
            passage = corpus[i]
            tokenized_examples = tokenizer(
                question,
                passage,
                truncation="only_second",
                max_length=512,
                stride=128,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                #return_token_type_ids=False,  # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
                padding="max_length",
                return_tensors='pt'
            )

            score = 0
            for i in range(len(tokenized_examples['input_ids'])) :
                c_input = {
                    'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
                    'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
                    'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')
                }
                tmp_score = c_encoder(**c_input).to('cpu')
                score += tmp_score
            score = score / len(tokenized_examples['input_ids'])
            question_score.append(score)

        sort_result = torch.sort(torch.tensor(score_list), descending=True)
        scores, index_list = sort_result[0], sort_result[1]

        result_scores.append(scores)
        result_indices.append(index_list)

<ipython-input-32-a97d169c2490>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
<ipython-input-32-a97d169c2490>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
<ipython-input-32-a97d169c2490>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')


KeyboardInterrupt: 

In [ ]:
top_k_index_list = []
for i in range(len(index_list)) :
    temp = index_list[i][:k]
    top_k_index_list.appedn(temp)

In [ ]:
total = []
for idx, example in enumerate(
        tqdm(dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": top_k_index_list[idx],
            "context": " ".join(  # 기존에는 ' '.join()
                [corpus[pid] for pid in top_k_index_list[idx]]
            ),
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        total.append(tmp)

cqas_100 = pd.DataFrame(total)